In [ ]:
import hydra
import pytorch_lightning as pl
import torch
from pathlib import Path
import torch.nn as nn

In [ ]:
inference_cfg = [
    [[], "efficientdet_d4_0.639.ckpt" , 'predictions_efficientdet_d4.json'],
    # [['module=retinanet'], "retinanet_resnet50_0.661.ckpt", "predictions_retinanet_resnet50.json"],
    # [['module=retinanet'], "retinanet_resnet50_0.661.ckpt", "predictions_retinanet_swint.json"]
]


In [ ]:
ckpt_name = "predictions_retinanet_swint.json"

In [ ]:
# create_overrides(ckpt_name)

In [ ]:
weights_root = Path('/datagrid/personal/kuntluka/weights').rglob("**/*.ckpt")

In [ ]:
str(f[0])

In [ ]:
f =list(weights_root)

In [ ]:
len(f)

In [ ]:
def create_overrides_and_path(ckpt_name):
    overrides = []
    target_path = ""
    if '/retinanet/' in ckpt_name:
        overrides.append('module=retinanet')
        target_path = Path('retinanet')

    elif '/faster_rcnn/' in ckpt_name:
        overrides.append('module=faster_rcnn')
        target_path = Path('faster_rcnn')

    elif '/yolov5/' in ckpt_name:
        overrides.append('module=yolov5')
        target_path = Path('yolov5')
    
    elif '/effdet/' in ckpt_name:
        overrides.append('module=efficientdet')
        target_path = Path('effdet')
    
    if '/medium_p6/' in ckpt_name:
        overrides.append('module.backbone=medium_p6')
        target_path = target_path / "medium"

    if '/small_p6/' in ckpt_name:
        overrides.append('module.backbone=small_p6')
        target_path = target_path / "small"

    elif '/large_p6/' in ckpt_name:
        overrides.append('module.backbone=large_p6')
        target_path = target_path / "large"

    elif '/resnet101/' in ckpt_name:
        overrides.append('module.backbone=resnet101_fpn_1x')
        target_path = target_path / "resnet101"

    elif '/resnet50/' in ckpt_name:
        overrides.append('module.backbone=resnet50_fpn_1x')
        target_path = target_path / "resnet50"

    elif '/swint/' in ckpt_name:
        overrides.append('module.backbone=swin_t_p4_w7_fpn_1x_coco')
        target_path = target_path / "swint"

    elif '/d0/' in ckpt_name:
        overrides.append('module.backbone=d0')
        target_path = target_path / "d0"
    elif '/d1/' in ckpt_name:
        overrides.append('module.backbone=d1')
        target_path = target_path / "d1"
    elif '/d2/' in ckpt_name:
        overrides.append('module.backbone=d2')
        target_path = target_path / "d2"
    elif '/tf_d0/' in ckpt_name:
        overrides.append('module.backbone=tf_d0')
        target_path = target_path / "tf_d0"
    elif '/tf_d3/' in ckpt_name:
        overrides.append('module.backbone=tf_d3')
        target_path = target_path / "tf_d3"

    return overrides, target_path

In [ ]:
def create_predictions_name(ckpt_name):
    base_name = ckpt_name.split('_0.')[0]
    name = 'predictions_' + base_name + '.json'
    return name

In [ ]:
overrides, name = create_overrides_and_path(str(f[0]))
overrides.append('logger.wandb.project=inference')
with hydra.initialize(config_path="configs"):
    cfg = hydra.compose(
        config_name="train",
        overrides=overrides
    )
model = hydra.utils.instantiate(cfg.module.model)
model.load_state_dict(torch.load(f[0], map_location='cpu')['state_dict'])

In [ ]:
logger = []
if "logger" in cfg:
    for _, lg_conf in cfg.logger.items():
        if "_target_" in lg_conf:
            logger.append(hydra.utils.instantiate(lg_conf))

In [ ]:
import albumentations as A
from src.transforms.albumentations_adapter import Adapter
statistics = dict(mean=0.3669, std=0.2768)
trainer = hydra.utils.instantiate(cfg.trainer, logger=logger)
composer = hydra.utils.instantiate(cfg.transforms, _recursive_=False)
t, v = composer.train_val_transforms()
v = Adapter([A.Resize(896,1024), A.Normalize(mean=statistics["mean"], std=statistics["std"])])
dm = hydra.utils.instantiate(cfg.datamodule, train_transforms=v, val_transforms=v)
dm.setup()

In [ ]:
test_prediction = trainer.predict(model, dm.predict_dataloader('test'))
val_prediction = trainer.predict(model, dm.predict_dataloader('val'))
train_prediction = trainer.predict(model, dm.predict_dataloader('train'))

In [ ]:
from src.utils.bbox_inverse_transform import predictions_to_fiftyone
test_data = predictions_to_fiftyone(test_prediction, stage='test')
val_data = predictions_to_fiftyone(val_prediction, stage='val')
train_data = predictions_to_fiftyone(train_prediction, stage='train')
data = {**test_data, **val_data, **train_data}

# import json
# with open(pred_name, 'w') as f:
#     json.dump(data, f)

In [ ]:
import json
from src.utils.conver_to_coco import to_coco
from src.evaluation.prediction_evaluation import PredictionEval

with open('metadata4000.json', 'r') as f:
    meta_data = json.load(f)

In [ ]:
ann_path = '/datagrid/personal/kuntluka/dental_rtg3/annotations.json'

with open(ann_path, 'r') as f:
    ann_file = json.load(f)

In [ ]:
data_path = Path('predictions/retinanet/swint/0.723.json')
with open(data_path, 'r') as f:
    data = json.load(f)

preds_coco, names = to_coco(data, ann_file)
pred_eval = PredictionEval()
pred_eval.load_data_coco_files(ann_path, preds_coco, names)
pred_eval.get_data('')

In [ ]:
yolo_L
(' & 0.347& 0.796 & 0.725 & 0.291 & 0.597 & 0.772 & 0.753 \\ \\hline',
 '0.689 & 0.69 & 0.689 & 0.271',
 ' & 0.559& 0.895 & 0.956 & 0.559 & 0.916 & 0.971 & 0.971 \\ \\hline')
yolo_m
(' & 0.343& 0.795 & 0.719 & 0.287 & 0.636 & 0.752 & 0.785 \\ \\hline',
 '0.74 & 0.64 & 0.686 & 0.326',
 ' & 0.547& 0.899 & 0.957 & 0.541 & 0.933 & 0.971 & 0.971 \\ \\hline')
yolo_s
(' & 0.327& 0.79 & 0.697 & 0.281 & 0.559 & 0.739 & 0.826 \\ \\hline',
 '0.692 & 0.64 & 0.665 & 0.295',
 ' & 0.545& 0.877 & 0.951 & 0.549 & 0.916 & 0.968 & 0.968 \\ \\hline')
effdet d1
(' & 0.319& 0.787 & 0.701 & 0.251 & 0.584 & 0.752 & 0.808 \\ \\hline',
 '0.739 & 0.64 & 0.686 & 0.339',
 ' & 0.531& 0.87 & 0.958 & 0.488 & 0.909 & 0.978 & 0.978 \\ \\hline')
faster 50
(' & 0.311& 0.788 & 0.705 & 0.231 & 0.629 & 0.737 & 0.788 \\ \\hline',
 '0.719 & 0.66 & 0.688 & 0.759',
 ' & 0.475& 0.867 & 0.883 & 0.445 & 0.832 & 0.899 & 0.899 \\ \\hline')
faster 101
(' & 0.316& 0.792 & 0.688 & 0.239 & 0.563 & 0.732 & 0.793 \\ \\hline',
 '0.704 & 0.63 & 0.665 & 0.684',
 ' & 0.478& 0.864 & 0.89 & 0.458 & 0.815 & 0.917 & 0.917 \\ \\hline')
retina swint
(' & 0.325& 0.803 & 0.723 & 0.249 & 0.579 & 0.78 & 0.758 \\ \\hline',
 '0.681 & 0.69 & 0.685 & 0.367',
 ' & 0.508& 0.89 & 0.946 & 0.468 & 0.902 & 0.967 & 0.967 \\ \\hline')

In [ ]:
map50 = pred_eval.map_query(stage='test')

In [ ]:
map50 = round(map50,3)

In [ ]:
name / (str(map50) + '.json')

In [ ]:
overrides, ckpt, pred_name = inference_cfg[1]
chckpt_path = Path("/datagrid/personal/kuntluka/pretrained/dataset_3500")
with hydra.initialize(config_path="configs"):
    cfg = hydra.compose(
        config_name="train",
        # overrides=overrides
        # overrides=['module=faster_rcnn', 'logger.wandb.project=inference']
        # overrides=['module=faster_rcnn', 'logger.wandb.project=inference', 'module.backbone=resnet101_fpn_1x']
        # overrides=['module=yolov5', 'logger.wandb.project=inference', 'module.backbone=medium_p6']
        # overrides=['module=yolov5', 'logger.wandb.project=inference', 'module.backbone=large_p6']
    )

model = hydra.utils.instantiate(cfg.module.model)

In [ ]:
# ckpt_name = "faster_rcnn_resnet101_map0.685.ckpt"
# ckpt_name = "faster_rcnn_resnet50_map0.689.ckpt"
# ckpt_name = "yolov5_medium_0.670.ckpt"
ckpt_name = "yolov5_large_0.648.ckpt"
ckpt_name = "efficientdet_d4_0.639.ckpt"
# ckpt_name = "efficientdet_d4_0.639.ckpt"
# ckpt_name=ckpt
model.load_state_dict(torch.load(chckpt_path/ckpt_name, map_location='cpu')['state_dict'])

In [ ]:
logger = []
if "logger" in cfg:
    for _, lg_conf in cfg.logger.items():
        if "_target_" in lg_conf:
            logger.append(hydra.utils.instantiate(lg_conf))

In [ ]:
import albumentations as A
from src.transforms.albumentations_adapter import Adapter
statistics = dict(mean=0.3669, std=0.2768)
trainer = hydra.utils.instantiate(cfg.trainer, logger=logger)
composer = hydra.utils.instantiate(cfg.transforms, _recursive_=False)
t, v = composer.train_val_transforms()
v = Adapter([A.Resize(896,1024), A.Normalize(mean=statistics["mean"], std=statistics["std"])])
dm = hydra.utils.instantiate(cfg.datamodule, train_transforms=v, val_transforms=v)
dm.setup()

In [ ]:
test_prediction = trainer.predict(model, dm.predict_dataloader('test'))
val_prediction = trainer.predict(model, dm.predict_dataloader('val'))
train_prediction = trainer.predict(model, dm.predict_dataloader('train'))

In [ ]:
# from src.utils.bbox_inverse_transform import predictions_to_fiftyone
from importlib import reload
import src.utils.bbox_inverse_transform
reload(src.utils.bbox_inverse_transform)
test_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(test_prediction, stage='test')
val_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(val_prediction, stage='val')
train_data = src.utils.bbox_inverse_transform.predictions_to_fiftyone(train_prediction, stage='train')
data = {**test_data, **val_data, **train_data}

import json
with open(pred_name, 'w') as f:
    json.dump(data, f)